# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [47]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

#1. Criar a instância de navegação
navegador = webdriver.Firefox()
navegador.get("https://www.google.com/")

#2. Encontrar os elementos com Xpath e guardar o valor
    # a- Acessar o site
    # a- Digitar a cotação no campo
    # b- Pressionar enter
    # c- Pegar o atributo com o valor da cotação
    # d- Repete
    
# -----[Dólar]----------------
navegador.find_element("xpath", 
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
navegador.find_element("xpath", 
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

time.sleep(1)
dolar = navegador.find_element('css selector', '.SwHCTb').get_attribute('data-value')
print(dolar)

# -----[Euro]----------------
navegador.get("https://www.google.com/")
navegador.find_element("xpath", 
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element("xpath", 
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

time.sleep(1)
euro = navegador.find_element('css selector', '.SwHCTb').get_attribute('data-value')
print(euro)

# -----[Ouro]---------------
navegador.get("https://www.melhorcambio.com/ouro-hoje")
ouro = navegador.find_element('xpath', 
    '//*[@id="comercial"]').get_attribute('value').replace(',', '.')
print(ouro)

# Pronto
navegador.quit()

5.1728
5.2900673760000005
293.37


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [45]:
import pandas as pd

#3. Importar a base e atualizar as colunas
    #a) Atualizar a cotação para cada moeda
    #b) Recalcular as colunas de preço de compra e venda
tabela = pd.read_excel("Produtos.xlsx")

# Localize as linhas que atendem à condição e substitua o valor da coluna cotação
tabela.loc[tabela["Moeda"]=='Dólar', "Cotação"] = float(dolar)
tabela.loc[tabela["Moeda"]=='Euro', "Cotação"] = float(euro)
tabela.loc[tabela["Moeda"]=='Ouro', "Cotação"] = float(ouro)

# Calcule o preço de compra e venda com essa nova cotação
    # Compra = Original * Cotação
    # Venda  = Compra * Margem
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]
tabela["Preço de Venda"]  = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.172800,5172.748272,1.40,7241.847581
1,Carro Renault,4500.00,Euro,5.290067,23805.303192,2.00,47610.606384
2,Notebook Dell,899.99,Dólar,5.172800,4655.468272,1.70,7914.296062
3,IPhone,799.00,Dólar,5.172800,4133.067200,1.70,7026.214240
4,Carro Fiat,3000.00,Euro,5.290067,15870.202128,1.90,30153.384043
5,Celular Xiaomi,480.48,Dólar,5.172800,2485.426944,2.00,4970.853888
6,Joia 20g,20.00,Ouro,293.370000,5867.400000,1.15,6747.510000


### Agora vamos exportar a nova base de preços atualizada

In [46]:
# 4. Exportar a base
tabela.to_excel("Produtos-novo.xlsx", index=False)